# 10 - Dataset Transforms

# 1. Custom Dataset

In [118]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision
import numpy as np
import math

In [119]:
class WineDataset(Dataset):
    def __init__(self, transform=None):
        super(WineDataset, self).__init__()
        xy = np.loadtxt("wine.txt", delimiter=",", dtype=np.float32, skiprows=1)
        # self.x = torch.tensor(xy[:, 1:], dtype=torch.float32)
        # self.y = torch.tensor(xy[:, [0]], dtype=torch.float32)
        self.x = xy[:, 1:]
        self.y = xy[:, [0]]
        
        self.n_samples = self.x.shape[0]
        
        self.transform = transform
    
    def __getitem__(self, index):
        sample = self.x[index], self.y[index]
        if self.transform:
            sample = self.transform(sample)
        
        return sample

    def __len__(self):
        return self.n_samples

# 2. Custom Transform

## 注意：有很多种Transform，自己去documentation查

## 2.1 ToTensor tansform

In [120]:
class ToTensor:
    def __call__(self, sample):
        inputs, targets = sample
        return torch.from_numpy(inputs), torch.from_numpy(targets)

In [121]:
dataset = WineDataset(transform=ToTensor())

In [122]:
first_data = dataset[0]
features, label = first_data
print(type(features), type(label))

<class 'torch.Tensor'> <class 'torch.Tensor'>


## 2.2 Multiple transform

In [123]:
class MulTransform:
    def __init__(self, factor):
        self.factor = factor
    
    def __call__(self, sample):
        inputs, target = sample
        inputs *= self.factor
        return inputs, target

In [124]:
dataset = WineDataset(transform=MulTransform(3))
first_data = dataset[0]
print(features)
features, label = first_data
print(type(features), type(label))

tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
        3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
        1.0650e+03])
<class 'numpy.ndarray'> <class 'numpy.ndarray'>


## 2.3 Composed transform

- `torchvision.transforms.Composed([Transform_1, Transform_2])`

In [127]:
composed = torchvision.transforms.Compose([ToTensor(), MulTransform(2)])
dataset = WineDataset(transform=None)
first_data = dataset[0]
features, label = first_data
print(features)
print(type(features), type(labels))

[1.423e+01 1.710e+00 2.430e+00 1.560e+01 1.270e+02 2.800e+00 3.060e+00
 2.800e-01 2.290e+00 5.640e+00 1.040e+00 3.920e+00 1.065e+03]
<class 'numpy.ndarray'> <class 'torch.Tensor'>


In [128]:
composed = torchvision.transforms.Compose([ToTensor(), MulTransform(2)])
dataset = WineDataset(transform=composed)
first_data = dataset[0]
features, label = first_data
print(features)
print(type(features), type(labels))

tensor([2.8460e+01, 3.4200e+00, 4.8600e+00, 3.1200e+01, 2.5400e+02, 5.6000e+00,
        6.1200e+00, 5.6000e-01, 4.5800e+00, 1.1280e+01, 2.0800e+00, 7.8400e+00,
        2.1300e+03])
<class 'torch.Tensor'> <class 'torch.Tensor'>
